In [13]:
from pathlib import Path
import simplekml
import numpy as np

In [14]:
coords = Path('data/handpick_train_coords.csv')
kml = simplekml.Kml()
with coords.open() as f:
    lls = f.read().split()
    lls = [float(ll[:-1]) for ll in lls]
lls = np.array(lls).reshape(-1, 2)
lls

array([[ -17.045093, -111.711268],
       [ -17.28878 , -111.588358],
       [ -16.904686, -111.620035],
       [ -17.323153, -111.08936 ],
       [ -17.120429, -111.095832],
       [ -17.650116, -112.092793],
       [ -17.529504, -112.500056],
       [ -18.051034, -112.708665],
       [ -18.049371, -112.651207],
       [ -18.519601, -111.202388],
       [ -18.979904, -110.570879],
       [ -17.958249, -111.757367],
       [ -17.656078, -110.519189],
       [ -18.111239, -110.462386],
       [ -17.846521, -111.424642],
       [ -17.705062, -111.531658],
       [ -17.871112, -111.035003],
       [ -18.012418, -111.980564],
       [ -16.976796, -112.492191]])

In [16]:
names = []
for ind, ll in enumerate(lls):
    kml.newpoint(coords=[(ll[1], ll[0])], name=f'mh_{ind + 1}')
    names.append(f'mh_{ind + 1}')
kml.save('out/handpick_test_coords.kml')

In [17]:
import pandas as pd

df = pd.DataFrame({'name': names, 'lat': lls[:, 0], 'lon': lls[:, 1]})
df.to_csv('out/handpick_test_coords.csv', index=False)

In [21]:
from smount_predictors import SeamountHelp
new_coords = pd.read_csv('out/handpick_test_coords.csv')
old_coords = pd.read_csv('data/all.xyhrdnc', sep=' ', names=['lon', 'lat', 'height', 'radius', 'dept', 'name', 'cat'])
old_coords = old_coords[['lat', 'lon', 'radius', 'name']]
points = SeamountHelp.readKMLbounds(Path('data/seamount_training_zone.kml'))
old_coords = old_coords[(old_coords['lat'] > points[0]) & (old_coords['lat'] < points[1]) & (old_coords['lon'] > points[2]) & (old_coords['lon'] < points[3])]
train_coords = pd.concat([old_coords, new_coords], ignore_index=True)
train_coords.to_csv('out/train_mounts.csv')